In [1]:
import numpy as np
import math
from bounding_box_utils.bounding_box_utils import iou, convert_coordinates
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy, match_multi
import tensorflow as tf
from keras import backend as K
import random

Using TensorFlow backend.


In [2]:
sess = tf.Session()
K.set_session(sess)
K.clear_session()

In [3]:
gt = np.load("outputs/predder.npy")

In [4]:
gt1 = gt[1]['predictions_1']
gt2 = gt[1]['predictions_2']
gt1_proj = gt[1]['predictions_1_proj_tot']
gt2_proj = gt[1]['predictions_2_proj_tot']

In [5]:
pred_1 = np.load("outputs/predictions_1_14.npy")
pred_1_proj = np.load("outputs/predictions_1_proj_14.npy")
pred_2 = np.load("outputs/predictions_2_14.npy")
pred_2_proj = np.load("outputs/predictions_2_proj_14.npy")

labels:  (5, 4)
y_encoded:  (17292, 4)
similarities:  (5, 17292)
bipartite_matches:  (5,)
similarities after bipartite:  (5, 17292)
matches:  (53,)  -  (53,)
similarities after matches:  (5, 17292)

In [27]:
y_true_1 = gt1_proj[:,:,:18]
y_pred_1 = pred_1_proj[:,:,18:]
y_true_2 = gt1_proj[:,:,18:]
y_pred_2 = pred_1_proj[:,:,18:]

In [7]:
batch_size = tf.shape(y_pred_1)[0]

In [28]:
y_true_1d = tf.convert_to_tensor(np.concatenate([y_true_1,y_true_1]),dtype=tf.float32)
y_pred_1d = tf.convert_to_tensor(np.concatenate([y_pred_1,y_pred_1]),dtype=tf.float32)
y_true_2d = tf.convert_to_tensor(np.concatenate([y_true_2,y_true_2]),dtype=tf.float32)
y_pred_2d = tf.convert_to_tensor(np.concatenate([y_pred_2,y_pred_2]),dtype=tf.float32)

In [32]:
batch_size = 1

def gt_rem(pred, gt):
    val = tf.subtract(tf.shape(pred)[1], tf.shape(gt)[1],name="gt_rem_subtract")
    gt = tf.slice(gt, [0, 0, 0], [1, tf.shape(pred)[1], 18],name="rem_slice")
    return gt

def gt_add(pred, gt):
    #add to gt
    val = tf.subtract(tf.shape(pred)[1], tf.shape(gt)[1],name="gt_add_subtract")
    ext = tf.slice(gt, [0, 0, 0], [1, val, 18], name="add_slice")
    gt = K.concatenate([ext,gt], axis=1)
    return gt

def equalalready(gt, pred): return pred

def make_equal(pred, gt):
    equal_tensor = tf.cond(tf.shape(pred)[1] < tf.shape(gt)[1], lambda: gt_rem(pred, gt), lambda: gt_add(pred, gt), name="make_equal_cond")
    return equal_tensor

def matcher(y_true_1,y_true_2,y_pred_1,y_pred_2, bsz):
    pred = 0
    gt = 0
    for i in range(bsz):
        filterer = tf.where(tf.not_equal(y_true_1[i,:,-1],99))
        y_true_new = tf.gather_nd(y_true_1[i,:,:],filterer)
        y_true_new = tf.expand_dims(y_true_new, 0)
        iou_out = tf.py_func(iou, [y_true_new[i,:,-16:-12],tf.convert_to_tensor(y_true_1[i,:,-16:-12])], tf.float64, name="iou_out")
        bipartite_matches = tf.py_func(match_bipartite_greedy, [iou_out], tf.int64, name="bipartite_matches")
        out = tf.gather(y_pred_2[i,:,:], [bipartite_matches], axis=0, name="out")
        
        filterer_2 = tf.where(tf.not_equal(y_true_2[i,:,-1],99))
        y_true_2_new = tf.gather_nd(y_true_2[i,:,:],filterer_2)
        y_true_2_new = tf.expand_dims(y_true_2_new, 0)

        box_comparer = tf.reduce_all(tf.equal(tf.shape(out)[1], tf.shape(y_true_2_new)[1]), name="box_comparer")
        y_true_2_equal = tf.cond(box_comparer, lambda: equalalready(out, y_true_2_new), lambda: make_equal(out, y_true_2_new), name="y_true_cond")
        if i != 0:
            pred = K.concatenate([pred,out], axis=-1)
            gt = K.concatenate([gt,y_true_2_equal], axis=0)
        else:
            pred = out
            gt = y_true_2_equal
    return pred, gt

In [33]:
y_pred, y_true = matcher(y_true_1d,y_true_2d,y_pred_1d,y_pred_2d,1)



In [34]:
matcher(y_true_1d,y_true_2d,y_pred_1d,y_pred_2d,1)

(<tf.Tensor 'out_9:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'y_true_cond_9/Merge:0' shape=(1, ?, 18) dtype=float32>)

In [26]:
t = tf.convert_to_tensor(y_true_1d, dtype=tf.float32)

ValueError: Tensor conversion requested dtype float32 for Tensor with dtype float64: 'Tensor("Const:0", shape=(2, 17292, 18), dtype=float64)'

In [13]:
x = tf.where(tf.not_equal(t[0,:,-1],99))

In [14]:
f = tf.gather_nd(t[0,:,:],x)



In [15]:
f = tf.expand_dims(f, 0)

In [16]:
K.eval(f).shape


(1, 4, 18)